<a href="https://colab.research.google.com/github/WhiteCJbr/DeepLearning/blob/main/Classificacao_com_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports


In [11]:
import tensorflow
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model

Carregando o Dataset

In [2]:
dataset = keras.datasets.fashion_mnist
((imagens_treino, identificacoes_treino), (imagens_teste, identificacoes_teste)) = dataset.load_data() # ler os dados, devolve 2 tuplas

Exploração dos Dados


In [30]:
len(imagens_treino)
imagens_treino.shape # 60000 imagens de tamanho 28 x 28
imagens_teste.shape
len(identificacoes_teste)

identificacoes_treino.min() # 0
identificacoes_treino.max() # 9
total_de_classificacoes = 10

(10000, 28, 28)

Exibição dos Dados

In [ ]:
nomes_de_classificacoes = ['Camiseta', 'Calca', 'Pullover', 'Vestido', 'Casaco', 'Sandalia', 'Camisa', 'Tenis', 'Bolsa', 'Bota']

# plt.imshow(imagens_treino[0])
# plt.title(identificacoes_treino[0])

# for imagem in range(10):
#   plt.subplot(2, 5, imagem+1)
#   plt.imshow(imagens_treino[imagem])
#   plt.title(nomes_de_classificacoes[identificacoes_treino[imagem]])

plt.imshow(imagens_treino[0])
plt.colorbar()

# Normalizando as Imagens


In [ ]:
imagens_treino = imagens_treino/float(255)

# Construção da Rede , Compilando, Treinando e Normalizadndo o Modelo

In [ ]:
modelo = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),                         # Tamanho da entrada, no caso das imagens, elas possuem 28px x 28px -> Achatamos essa entrada para um array de 1 dimensao
    keras.layers.Dense(256, activation=tensorflow.nn.relu),             # Camada hidden, totalmente conectada com a primeira camada (entrada), com ativação ReLU
    keras.layers.Dropout(0.2),                                          # -> Tratamento de Overfitting, é um tipo de normalização
    # keras.layers.Dense(128, activation=tensorflow.nn.relu),
    # keras.layers.Dense(64, activation=tensorflow.nn.relu),            # Nesse caso, essas camadas intermediárias ferraram com o cu do conde, então não vamos usar
    keras.layers.Dense(10, activation=tensorflow.nn.softmax)            # 10 saídas
])

modelo.compile(optimizer='adam',                       # Precisamos compilar o modelo antes de treina-lo modelo.compile(optimizer, loss)
               loss='sparse_categorical_crossentropy', # Definimos o loss
               metrics=['accuracy'])                   # Definimos uma métrica para a acurácia

historico = modelo.fit(imagens_treino, identificacoes_treino, epochs=10, validation_split=0.2) # (dados_treino, saídas_esperadas, epocas, % para validação )

# Salvar e Carregar o Modelo Treinado

In [ ]:
modelo.save('modelo.h5')
modelo_salvo = load_model('modelo.h5')

# Gráfico de Acurácia dos treinos e validação por Época

In [ ]:
plt.plot(historico.history['accuracy'])
plt.plot(historico.history['val_accuracy'])
plt.title('Acurácia por épocas')
plt.xlabel('épocas')
plt.ylabel('acurácia')

plt.legend(['treino', 'avaliação'])

# Gráfico de Perda dos treinos e validação por Época

In [ ]:
plt.plot(historico.history['loss'])
plt.plot(historico.history['val_loss'])
plt.title('Perda por épocas')
plt.xlabel('épocas')
plt.ylabel('perda')

plt.legend(['treino', 'validação'])

## Como Interpretar os gráficos de acurácia

* Se a linha de treino está super bem e a linha e validação está abaixo do estimado, seu modelo está em **OVERFITTING**
  * Sua rede está entendendo muito mais o treino do que a validação

## Como interpretar os gráficos de Perda

* Caso seu gráfico de perda para os testes estejam muito mais baixos do que os de validação, seu modelo está em **OVERFITTING**
  * Sua rede está entendendo muito mais o treino do que a validação

## COMO FAZER O TRATAMENTO DO OVERFITTING

1. Análise do gráfico: verifique onde este gráfico está melhor em relação a perdas e acurácia, treine somente até onde ele está bom
2. Modifique a camada hidden do seu modelo: Adicione uma camada de Dropout() para que seus neurônios não percam tanta informação no treinamento

# Testando o modelo e o modelo_salvo

In [7]:
testes = modelo.predict(imagens_teste) # Fazer os testes para esse modelo criado
print('Resultado Teste:  ',np.argmax(testes[2])) # Achar o maior valor (resposta da rede) usando o numpy
print('Numero da imagem de teste: ', identificacoes_teste[2])

testes_modelo_salvo = modelo_salvo.predict(imagens_teste)
print('Resultado Teste Modelo_Salvo:  ',np.argmax(testes_modelo_salvo[2]))
print('Numero da imagem de teste: ', identificacoes_teste[2])

313/313 [==============================] - 1s 3ms/step
Resultado Teste:   1
Numero da imagem de teste:  1
313/313 [==============================] - 1s 3ms/step
Resultado Teste Modelo_Salvo:   1
Numero da imagem de teste:  1


#Avaliando o Modelo


In [8]:
perda_teste, acuracia_teste = modelo.evaluate(imagens_teste, identificacoes_teste)
print('Perda do teste: ', perda_teste)
print('Acurácia do teste: ', acuracia_teste)

313/313 [==============================] - 1s 2ms/step - loss: 53.6032 - accuracy: 0.8654
Perda do teste:  53.60319900512695
Acurácia do teste:  0.8654000163078308
